In [2]:
import csv
import os
import requests as req
import json
from collections import Counter

In [3]:
#Donorschoose.org API
api_key = "DONORSCHOOSE"
url = "https://api.donorschoose.org/common/json_feed.html?max=100"

In [4]:
#Open a CSV for a list of Bay Area cities
with open("bayarea_cities.csv") as cities:
    cities_reader = csv.reader(cities)
    
    cities = [city[0] for city in cities_reader]
    
    print(len(cities))

cities = cities[57:69]
print(cities)

102
['Oakland', 'Oakley', 'Orinda', 'Pacifica', 'Palo Alto', 'Petaluma', 'Piedmont', 'Pinole', 'Pittsburg', 'Pleasant Hill', 'Pleasanton', 'Portola Valley']


In [5]:
# Build partial query URL
query_url = url + "&APIkey=" + api_key + "&state=CA" + "&cityName="
#location_data = [req.get(query_url + city).json() for city in cities]

all_data = {}

for city in cities:
    
    results = req.get(query_url + city).json()
    all_data[city] = results   

print(json.dumps(all_data, indent=2))

{
  "Oakland": {
    "searchTerms": "CA",
    "searchURL": "https://www.donorschoose.org/donors/search.html?APIkey=DONORSCHOOSE&cityName=Oakland&state=CA&utm_source=api&utm_medium=feed&utm_content=searchlink&utm_campaign=DONORSCHOOSE",
    "totalProposals": "219",
    "index": "0",
    "max": "50",
    "breadcrumb": [
      [
        "max",
        "100",
        ""
      ],
      [
        "state",
        "CA",
        "California"
      ]
    ],
    "proposals": [
      {
        "id": "2965406",
        "hashCode": "1619492249",
        "proposalURL": "https://www.donorschoose.org/project/improving-literacy-with-home-language-re/2965406/?verify=1619492249&utm_source=api&utm_medium=feed&utm_content=bodylink&utm_campaign=DONORSCHOOSE",
        "fundURL": "https://secure.donorschoose.org/donors/givingCart.html?proposalid=2965406&donationAmount=45&verify=1619492249&utm_source=api&utm_medium=feed&utm_content=fundlink&utm_campaign=DONORSCHOOSE",
        "imageURL": "https://storage.donor

In [6]:
city_data = {}

for cityname, data in all_data.items():
        
    #step 1: get cost to complete
    #step 2: get povertyType label
    #step 3: add this to the output vairable
    
    costcomplete = []
    povertytype = []
    
    data["proposals"]
    
    for x in data["proposals"]:
        
        costcomplete.append(float(x["costToComplete"]))
        povertytype.append(x["povertyType"]["label"])
    
    city_data[cityname] = {
        
        "costToComplete": sum(costcomplete),
        "povertyType": Counter(povertytype)
    }

city_data

{'Oakland': {'costToComplete': 31096.3,
  'povertyType': Counter({'HIGH': 15,
           'HIGHEST': 25,
           'MODERATE': 2,
           'MODERATEHIGH': 7,
           'MODERATELOW': 1})},
 'Oakley': {'costToComplete': 44467.36,
  'povertyType': Counter({'MODERATE': 35,
           'MODERATEHIGH': 5,
           'MODERATELOW': 10})},
 'Orinda': {'costToComplete': 127.2, 'povertyType': Counter({'LOW': 1})},
 'Pacifica': {'costToComplete': 3554.33,
  'povertyType': Counter({'LOW': 1, 'UNKNOWN': 1})},
 'Palo Alto': {'costToComplete': 0, 'povertyType': Counter()},
 'Petaluma': {'costToComplete': 1135.49,
  'povertyType': Counter({'HIGH': 1, 'MODERATE': 1})},
 'Piedmont': {'costToComplete': 1305.48, 'povertyType': Counter({'LOW': 1})},
 'Pinole': {'costToComplete': 9483.550000000001,
  'povertyType': Counter({'MODERATE': 3,
           'MODERATEHIGH': 7,
           'MODERATELOW': 2})},
 'Pittsburg': {'costToComplete': 15808.61,
  'povertyType': Counter({'HIGH': 15, 'MODERATEHIGH': 1})},
 'P

In [7]:
#list of cityname, costtocomplete, number of low, number of modeatelow, etc.

csv_data = []

citynames = sorted(city_data.keys())

for cityname in citynames: 
    
    data = city_data[cityname]
    
    one_row = []
    
    one_row.append(cityname)
    one_row.append(data['costToComplete'])  
    
    labeltype = ["LOW", "MODERATELOW", "MODERATE", "MODERATEHIGH", "HIGH", "HIGHEST", "UNKNOWN"]
    
    for x in labeltype:
        
        one_row.append(data['povertyType'].get(x, 0))
    
    csv_data.append(one_row)

csv_data

[['Oakland', 31096.3, 0, 1, 2, 7, 15, 25, 0],
 ['Oakley', 44467.36, 0, 10, 35, 5, 0, 0, 0],
 ['Orinda', 127.2, 1, 0, 0, 0, 0, 0, 0],
 ['Pacifica', 3554.33, 1, 0, 0, 0, 0, 0, 1],
 ['Palo Alto', 0, 0, 0, 0, 0, 0, 0, 0],
 ['Petaluma', 1135.49, 0, 0, 1, 0, 1, 0, 0],
 ['Piedmont', 1305.48, 1, 0, 0, 0, 0, 0, 0],
 ['Pinole', 9483.550000000001, 0, 2, 3, 7, 0, 0, 0],
 ['Pittsburg', 15808.61, 0, 0, 0, 1, 15, 0, 0],
 ['Pleasant Hill', 19151.260000000002, 9, 15, 3, 0, 4, 0, 0],
 ['Pleasanton', 41440.310000000005, 39, 0, 0, 0, 0, 0, 0],
 ['Portola Valley', 0, 0, 0, 0, 0, 0, 0, 0]]

In [8]:
root_path = os.path.join(os.getcwd())
data_path = os.path.join(root_path)
csvpath = os.path.join(root_path, "donorschoose_api.csv")
with open(csvpath, "a", newline="") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=",")
    csvwriter.writerows(csv_data)